In [ ]:
from collections import defaultdict
import json

import numpy as np
import pandas as pd

import time

In [ ]:
rfpl_info_file = './Data/rfpl_info.json'
with open(rfpl_info_file, 'r') as file:
    data = json.load(file)

* Get the venues info:

In [ ]:
matches = data["data"]["stat_tournament"]["currentSeason"]["matches"]
VENUE_COLS = ['id', 'name', 'capacity', 'lon', 'lat']

In [ ]:
matches_info = defaultdict(list)
for match in matches:
    matches_info['match_id'].append(match['id'])
    matches_info['match_start_time'].append(match['scheduledAtStamp'])
    matches_info['home_team_id'].append(match['home']['team']['id'])
    matches_info['home_team_name'].append(match['home']['team']['name'])
    matches_info['home_team_logo'].append(match['home']['team']['logo']['main'])
    matches_info['away_team_id'].append(match['away']['team']['id'])
    matches_info['away_team_name'].append(match['away']['team']['name'])
    matches_info['away_team_logo'].append(match['away']['team']['logo']['main'])
    home_score = 0
    away_score = 0
    for scores in match['periodScore']:
        home_score += scores['homeScore']
        away_score += scores['awayScore']
    matches_info['home_score'].append(home_score)
    matches_info['away_score'].append(away_score)
    if match['venue'] is not None:
        for col in VENUE_COLS:
            matches_info['venue_'+col].append(match['venue'][col])
    else:
        for col in VENUE_COLS:
            matches_info['venue_'+col].append(None)

In [ ]:
MATCHES_INFO_PATH = './Data/matches_info.csv'
matches_df = pd.DataFrame(
    data=matches_info
)
matches_df.dropna(inplace=True)
matches_df.to_csv(MATCHES_INFO_PATH)
matches_df

* Match the User's location and time with the suitable match:

In [ ]:
def return_closest_match_info(input_dict, matches_db_path='./Data/matches_info.csv'):
    '''
    input_dict = {'lon': 0, 'lat': 0, 'time': unicode_time}
    '''
    matches_df = pd.read_csv(matches_db_path)
    lonlat = matches_df.loc[:, ['venue_lon', 'venue_lat']].values
    user_lonlat = np.array([input_dict['lon'], input_dict['lat']])
    dists = np.square(lonlat[:,np.newaxis]-user_lonlat).sum(axis=2)
    closest_venue_id = matches_df.iloc[np.argmin(dists),:]['venue_id']
    closest_matches = matches_df[matches_df['venue_id'] == closest_venue_id]
    closest_match_id = np.argmin(closest_matches['match_start_time'].values - input_dict['time'])
    result = dict(closest_matches.iloc[closest_match_id, :])
    INT_COLS = ['match_id', 'match_start_time', 'venue_capacity']
    FLOAT_COLS = ['venue_lon', 'venue_lat']
    for col in INT_COLS:
        result[col] = int(result[col])
    for col in FLOAT_COLS:
        result[col] = float(result[col])
    del result['Unnamed: 0']
    return result

In [ ]:
input_dict = {'lon': 32.0547, 'lat': 49.4361, 'time': 1564419600}                                              
out = return_closest_match_info(input_dict)

In [ ]:
out

In [ ]:
for col in out:
    print(type(out[col]))